In [1]:
from pptx import Presentation
from win32com import client
import win32com.client as win32
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import re
import sys 
win32c = win32.constants
import shutil
import os
from win32com.client import constants as xl
import warnings
warnings.filterwarnings("ignore")
import time

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x9
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 9) - <class 'ImportError'>: cannot import name 'Dispatch' from 'win32com.client' (c:\Users\BW4SA\AppData\Local\Programs\Python\Python37\lib\site-packages\win32com\client\__init__.py)
Done.


In [2]:
MODULE_LIST = [m.__name__ for m in sys.modules.values()]
for module in MODULE_LIST:
    if re.match(r'win32com\.gen_py\..+', module):
        del sys.modules[module]
directory_path = os.path.join(os.environ.get('LOCALAPPDATA'), 'Temp', 'gen_py')

# Check if the directory exists
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)

In [3]:
def filter_dictionary_keys(dictionary, substring):
    filtered_keys = {key: value for key, value in dictionary.items() if substring in key}
    return filtered_keys 

#Set the filter,column,rows and values for the excel sheet
def set_excel_fields(row_list=[],column_list=[],filter_list=[],value_list=[],pvtTable=''):
    for row in row_list:
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, row).values())[0]).Orientation = 1 #Rows 
    for col in column_list:
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, col).values())[0]).Orientation = 2 #Columns

    if '[Calendar].[Year]' in column_list:
        pvtTable.PivotFields('[Calendar].[Year].[Year]').ClearAllFilters()
        pvtTable.CubeFields('[Calendar].[Year]').EnableMultiplePageItems = True 
        visible_items = []
        for value in years:
            visible_items.append(f"[Calendar].[Year].&[{value}]")
        pvtTable.PivotFields('[Calendar].[Year].[Year]').VisibleItemsList = visible_items 
        
    for filters in filter_list:
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, filters).values())[0]).Orientation = 3 #Filter
    for value in value_list:
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, value).values())[0]).Orientation = 4 #Value 
    return pvtTable



def create_dataframe(list_filter,df,filter_name,pvtTable,name='',list_filter_2=[],filter_name_2=''):
    if len(list_filter_2)==0:
        for filters in list_filter:
            if name+filters in df.keys():
                continue
            pvtTable.PivotFields(filter_name).ClearAllFilters()
            pvtTable.PivotFields(filter_name).CurrentPageName = '.'.join(filter_name.split('.')[:2])+'.&['+filters+']'   
            wb.Save()
            con = pd.read_excel(f_name)        
            df[name+filters] =con
    else:
        for filters in list_filter:
            required_keys = [name+filters+' | '+ i for i in list_filter_2]
            if all(key in df.keys() for key in required_keys):
                continue
            pvtTable.PivotFields(filter_name).ClearAllFilters()
            pvtTable.PivotFields(filter_name).CurrentPageName = '.'.join(filter_name.split('.')[:2])+'.&['+filters+']'   
            for filters_2 in list_filter_2:
                if name+filters+' | '+filters_2 in df.keys():
                    continue
                pvtTable.PivotFields(filter_name_2).ClearAllFilters()
                pvtTable.PivotFields(filter_name_2).CurrentPageName = '.'.join(filter_name_2.split('.')[:2])+'.&['+filters_2+']'   
                wb.Save()
                con = pd.read_excel(f_name)        
                df[name+filters+' | '+filters_2] =con
                
    return df


def Market_Duplication_CreateDFs(area_name, regions, channels, markets, pvtTable, data_frame, name = '',list_filter_2=[],filter_name_2=''):
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = area_name
    if area_name == '[Market].[Area].&[NATIONAL]':
        if list_filter_2:
            data_frame = create_dataframe(list_filter_2, data_frame,filter_name_2, pvtTable,name+'National | ')      
        else:
            wb.Save()
            con = pd.read_excel(f_name)        
            data_frame[name+'National'] =con
            
    else:    
        if len(regions) != 0:
            data_frame = create_dataframe(regions, data_frame, "[Market].[Region].[Region]", pvtTable,name,list_filter_2,filter_name_2)
            pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()
            if len(list_filter_2)!=0:
                pvtTable.PivotFields(filter_name_2).ClearAllFilters()

        if len(channels) != 0:
            data_frame = create_dataframe(channels, data_frame, "[Market].[Channel].[Channel]", pvtTable, name,list_filter_2,filter_name_2)
            pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
            if len(list_filter_2)!=0:
                pvtTable.PivotFields(filter_name_2).ClearAllFilters()    
                
        if len(markets) != 0:
            data_frame = create_dataframe(markets, data_frame, "[Market].[Market].[Market]", pvtTable, name,list_filter_2,filter_name_2)
            pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()
            if len(list_filter_2)!=0:
                pvtTable.PivotFields(filter_name_2).ClearAllFilters()
                
    return data_frame

### Functions_used

In [34]:
#define areas and loop of regions&channels&markets 
def set_area_filter(pvtTable, area):
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{area}]'

    
def process_area_data(pvtTable, data_frame, area, regions, channels, market, name = '',loop = [], loop_column =''):
    set_area_filter(pvtTable, area)
    if len(regions) != 0:
        data_frame = create_dataframe(regions, data_frame, "[Market].[Region].[Region]", pvtTable, name,loop,loop_column)
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels) != 0:
        data_frame = create_dataframe(channels, data_frame, "[Market].[Channel].[Channel]", pvtTable, name, loop,loop_column)
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()

    if len(market) != 0:
        data_frame = create_dataframe(market, data_frame, "[Market].[Market].[Market]", pvtTable, name,loop,loop_column)
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()

    return data_frame

In [1]:
#category steps reptation
def calculate_category_data(data_frame,name_new = '',national=True):
    if "NATIONAL" in areas and national:
        set_area_filter(pvtTable, "NATIONAL")
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        data_frame[f"{categories[0]}"+name_new+' | National'] = con     
    if "RETAILER" in areas:
        data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,f"{categories[0]}"+ name_new +" | " )
    if "CHANNEL" in areas:
        data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,f"{categories[0]}"+ name_new +" | ")
    if customareas in areas:
        data_frame = process_area_data(pvtTable, data_frame,customareas, regions_CUST, channels_CUST, market_CUST,f"{categories[0]}"+ name_new +" | ")
    return data_frame

In [2]:
# #category steps reptation

# def calculate_category_data(data_frame,name_new = ''):
#     if "NATIONAL" in areas and national:
#         set_area_filter(pvtTable, "NATIONAL")
#         wb.Save()
#         con = pd.read_excel(f_name, sheet_name=s_name)        
#         data_frame[f'{categories[0]} | National' + name_new] = con

#     if "RETAILER" in areas:
#         data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,f"{categories[0]}"+ name_new +" | " )

#     if "CHANNEL" in areas:
#         data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,f"{categories[0]}"+ name_new +" | ")
#     if customareas in areas:
#         data_frame = process_area_data(pvtTable, data_frame,customareas, regions_CUST, channels_CUST, market_CUST,f"{categories[0]}"+ name_new +" | ")
#     return data_frame

In [ ]:
#sector&segment steps reptation

def calculate_sector_segment_data(data_frame,name_new = ''):
        # Dataframes by months by Sectors for Areas = NATIONAL, RETAILER, CHANNEL,Pos

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    if len(sectors) != 0:

        if "NATIONAL" in areas:
            set_area_filter(pvtTable, 'NATIONAL')
            data_frame = create_dataframe(sectors,data_frame,'[Products].[Sector].[Sector]',pvtTable, 'National | '+ name_new )
            pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()
            
        if "RETAILER" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,name_new,sectors,'[Products].[Sector].[Sector]')
            pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

        if "CHANNEL" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,name_new,sectors,'[Products].[Sector].[Sector]')
            pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

        if customareas in areas:
            data_frame = process_area_data(pvtTable, data_frame,f'{customareas}', regions_CUST, channels_CUST, market_CUST,name_new,sectors,'[Products].[Sector].[Sector]')
            pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()


    # Dataframes by months by Segments for Areas = NATIONAL, RETAILER, CHANNEL
    if len(segments) != 0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'

        if "NATIONAL" in areas:
            set_area_filter(pvtTable, 'NATIONAL')
            data_frame = create_dataframe(segments,data_frame,'[Products].[Segment].[Segment]',pvtTable, 'National | '+ name_new )
            pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

        if "RETAILER" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,name_new,segments,'[Products].[Segment].[Segment]')
            pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

        if "CHANNEL" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,name_new,segments,'[Products].[Segment].[Segment]')
            pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

        if customareas in areas:
            data_frame = process_area_data(pvtTable, data_frame,f'{customareas}', regions_CUST, channels_CUST, market_CUST,name_new,segments,'[Products].[Segment].[Segment]')
            pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()


    if len(subsegments) != 0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'

        if "NATIONAL" in areas:
            set_area_filter(pvtTable, 'NATIONAL')
            data_frame = create_dataframe(subsegments,data_frame,'[Products].[SubSegment].[SubSegment]',pvtTable, 'National | '+ name_new )
            pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

        if "RETAILER" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,name_new,subsegments,'[Products].[SubSegment].[SubSegment]')
            pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

        if "CHANNEL" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,name_new,subsegments,'[Products].[SubSegment].[SubSegment]')
            pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

        if customareas in areas:
            data_frame = process_area_data(pvtTable, data_frame,f'{customareas}', regions_CUST, channels_CUST, market_CUST,name_new,subsegments,'[Products].[SubSegment].[SubSegment]')
            pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()     


    if len(subcategories) != 0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'

        if "NATIONAL" in areas:
            set_area_filter(pvtTable, 'NATIONAL')
            data_frame = create_dataframe(subcategories,data_frame,'[Products].[SubCategory].[SubCategory]',pvtTable, 'National | '+ name_new )
            pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

        if "RETAILER" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'RETAILER', regions_RET, channels_RET, market_RET,name_new,subcategories,'[Products].[SubCategory].[SubCategory]')
            pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

        if "CHANNEL" in areas:
            data_frame = process_area_data(pvtTable, data_frame, 'CHANNEL', regions_CHAN, channels_CHAN, market_CHAN,name_new,subcategories,'[Products].[SubCategory].[SubCategory]')
            pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

        if customareas in areas:
            data_frame = process_area_data(pvtTable, data_frame,f'{customareas}', regions_CUST, channels_CUST, market_CUST,name_new,subcategories,'[Products].[SubCategory].[SubCategory]')
            pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()                            
        
    return data_frame 

In [ ]:
def calculate_sec_seg_subseg_subcat_only(data_frame):
    if len(categories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        data_frame[f'{categories[0]}'] = con   
        
    if len(sectors)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'

        data_frame = create_dataframe(sectors,data_frame,'[Products].[Sector].[Sector]',pvtTable)
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'

        data_frame= create_dataframe(segments,data_frame,'[Products].[Segment].[Segment]',pvtTable)
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()
    
    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'

        data_frame= create_dataframe(subsegments,data_frame,'[Products].[SubSegment].[SubSegment]',pvtTable)
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'

        data_frame= create_dataframe(subcategories,data_frame,'[Products].[SubCategory].[SubCategory]',pvtTable )
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    return data_frame    

In [ ]:
def calculate_sec_seg_subseg_subcat_Manufacturer(data_frame):
    
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList = [f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]",f"[Products].[{ManufOrTopC}].&[{client_manuf[1]}]"]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"


    if len(categories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        wb.Save()
        con = pd.read_excel(f_name, sheet_name=s_name)        
        data_frame[f'{categories[0]} | {client_manuf[0]}'] = con

    if len(sectors)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
        data_frame= create_dataframe(sectors,data_frame,'[Products].[Sector].[Sector]',pvtTable,f'{client_manuf[0]} | ' )
        pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
        data_frame= create_dataframe(segments,data_frame,'[Products].[Segment].[Segment]',pvtTable ,f'{client_manuf[0]} | ')
        pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()
    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
        data_frame= create_dataframe(subsegments,data_frame,'[Products].[SubSegment].[SubSegment]',pvtTable ,f'{client_manuf[0]} | ')
        pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()
    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
        data_frame= create_dataframe(subcategories,data_frame,'[Products].[SubCategory].[SubCategory]',pvtTable ,f'{client_manuf[0]} | ')
        pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    return data_frame    


In [ ]:
def calculate_sec_seg_subseg_subcat_Brands(data_frame):
    for brand in client_brands:
        pvtTable.PivotFields('[Products].[Top Brands].[Top Brands]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Top Brands].[Top Brands]').CurrentPageName = f'[Products].[Top Brands].&[{brand}]'
        
        if len(categories)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            data_frame[f'{categories[0]} | {brand}'] = con

        if len(sectors)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
            data_frame= create_dataframe(sectors,data_frame,'[Products].[Sector].[Sector]',pvtTable,f'{brand} | ' )
            pvtTable.PivotFields('[Products].[Sector].[Sector]').ClearAllFilters()

        if len(segments)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
            data_frame= create_dataframe(segments,data_frame,'[Products].[Segment].[Segment]',pvtTable ,f'{brand} | ')
            pvtTable.PivotFields('[Products].[Segment].[Segment]').ClearAllFilters()

        if len(subsegments)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
            data_frame= create_dataframe(subsegments,data_frame,'[Products].[SubSegment].[SubSegment]',pvtTable ,f'{brand} | ')
            pvtTable.PivotFields('[Products].[SubSegment].[SubSegment]').ClearAllFilters()

        if len(subcategories)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
            data_frame= create_dataframe(subcategories,data_frame,'[Products].[SubCategory].[SubCategory]',pvtTable ,f'{brand} | ')
            pvtTable.PivotFields('[Products].[SubCategory].[SubCategory]').ClearAllFilters()

    return data_frame

In [ ]:
def calculate_retailer_channel_market_Brands(data_frame):

    if "NATIONAL" in areas:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
        data_frame = create_dataframe(client_brands,data_frame,"[Products].[Top Brands].[Top Brands]",pvtTable,f'{categories[0]} | National | ')

    ###################################### Area = Retailer #########################################
    if "RETAILER" in areas: 
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

        if len(regions_RET) != 0:
            data_frame= create_dataframe(regions_RET,data_frame,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Region].[Region]").ClearAllFilters()
        if len(channels_RET) != 0:
            data_frame= create_dataframe(channels_RET,data_frame,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Channel].[Channel]").ClearAllFilters()
        if len(market_RET) != 0:
            data_frame = create_dataframe(market_RET,data_frame,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters() 

                ###################################### Area = Channel #########################################

    if "CHANNEL" in areas:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'
            
        if len(regions_CHAN) != 0:
            data_frame= create_dataframe(regions_CHAN,data_frame,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Region].[Region]").ClearAllFilters()
        if len(channels_CHAN) != 0:
            data_frame= create_dataframe(channels_CHAN,data_frame,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Channel].[Channel]").ClearAllFilters()
        if len(market_CHAN) != 0:
            data_frame = create_dataframe(market_CHAN,data_frame,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters() 

    ###################################### Area = CUSTOM #########################################
    if customareas in areas:
        pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'

        if len(regions_CUST) != 0:
            data_frame= create_dataframe(regions_CUST,data_frame,"[Market].[Region].[Region]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Region].[Region]").ClearAllFilters()
        if len(channels_CUST) != 0:
            data_frame= create_dataframe(channels_CUST,data_frame,"[Market].[Channel].[Channel]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields("[Market].[Channel].[Channel]").ClearAllFilters()
        if len(market_CUST) != 0:
            data_frame = create_dataframe(market_CUST,data_frame,"[Market].[Market].[Market]",pvtTable, f'{categories[0]} | ',client_brands,"[Products].[Top Brands].[Top Brands]")
            pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters() 

    return data_frame        


In [ ]:
def calculate_data(data_frame,name = '',loop_list = [], loop_column =''):
    if "NATIONAL" in areas and national==True:
        data_frame = Market_Duplication_CreateDFs('[Market].[Area].&[NATIONAL]',[], [], [], pvtTable, data_frame, name,loop_list,loop_column)
    if "RETAILER" in areas:
        data_frame = Market_Duplication_CreateDFs('[Market].[Area].&[RETAILER]',regions_RET, channels_RET, market_RET, pvtTable, data_frame, name,loop_list,loop_column )
    if "CHANNEL" in areas:
        data_frame = Market_Duplication_CreateDFs('[Market].[Area].&[CHANNEL]',regions_CHAN, channels_CHAN, market_CHAN, pvtTable, data_frame, name,loop_list,loop_column )
    if customareas in areas:
        data_frame = Market_Duplication_CreateDFs(f'[Market].[Area].&[{customareas}]', regions_CUST, channels_CUST, market_CUST, pvtTable, data_frame, name ,loop_list,loop_column)
    return data_frame